In [1]:
import pandas as pd
from itertools import islice
import os
import json

In [2]:
%%bash
curl http://localhost:11434/api/chat -d '{
  "model": "qwen2.5-coder:32b",
  "messages": [
    {
      "role": "user",
      "content": "who wrote the book godfather?"
    }
  ],
  "stream": false,
  "options": {
    "temperature": 5,
    "num_predict": 75
    }
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   734  100   511  100   223    131     57  0:00:03  0:00:03 --:--:--   188


{"model":"qwen2.5-coder:32b","created_at":"2024-12-11T06:52:26.392666Z","message":{"role":"assistant","content":"The book \"The Godfather\" was written by Mario Puzo. It was first published in 1969 and became an enormous success, leading to subsequent sequels and a famous film adaptation directed by Francis Ford Coppola."},"done_reason":"stop","done":true,"total_duration":3895614875,"load_duration":554263209,"prompt_eval_count":36,"prompt_eval_duration":689000000,"eval_count":49,"eval_duration":2479000000}

In [3]:
import requests
import json

url = "http://localhost:11434/api/chat"

/Users/ethan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# llama3('who wrote the book godfather?')

In [5]:
def batch_iterator(iterator, batch_size):
    iterator = iter(iterator)
    while True:
        batch = tuple(islice(iterator, batch_size))
        if not batch:
            return
        yield batch

In [6]:
def read_system_prompt(file_path):
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            return file.read()
    else:
        return ""

In [7]:
def read_completed_ids(output_csv_path):
    if os.path.exists(output_csv_path) and os.path.getsize(output_csv_path) > 0:
        df = pd.read_csv(output_csv_path)
        return set(df['id'])
    else:
        return set()

In [8]:
# from transformers import StoppingCriteria, StoppingCriteriaList
#
# class EndCurlyBraceStoppingCriteria(StoppingCriteria):
#     def __init__(self, tokenizer):
#         self.tokenizer = tokenizer
#         self.end_token_id = tokenizer.convert_tokens_to_ids("}")
#
#     def __call__(self, input_ids, scores, **kwargs):
#         # Check if the last token generated is the end curly brace
#         return input_ids[0, -1] == self.end_token_id

In [9]:
# !pip install compressed-tensors

In [10]:
!pip3 install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [11]:
# !pip install llmcompressor
# !pip install transformers[torch]

In [12]:
# !pip install compressed-tensors

In [13]:
# input_df['id'] = input_df.index.astype(str) +'_US' +  input_df['userId'].astype(str)
# input_df.head()
# input_df.to_csv('fin_tst.csv', index=False)

In [14]:
# !pip3 install transformers

In [15]:
def llama3(prompt):
    data = {
        "model": "llama3:70b",
        # "model": "qwen2.5-coder:32b",
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ],
        "stream": False,
        "options": {
            "temperature": 0.7,
            "top_p": 0.2,
            "num_predict": 75,
            "stop":["}"],
        },
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    # print('r: ', response.json())
    return response.json()["message"]["content"]


In [16]:
# llama3('Who shot mr. burns?')

In [17]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
from itertools import islice
# import torch

system_prompt_file='prompt'
input_csv_path= 'orig_data/fin_tst_10.csv'
batch_size=10

# Set device
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
#
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# print('Going into the main loop now!')

# end_curly_brace_criteria = EndCurlyBraceStoppingCriteria(tokenizer)
i = 0

In [18]:
import random
def generate_prompt(row, system_prompt):
    sample_string = ''
    for key, value in row['samples'].items():
        sample_string += f'{key}: {value}'

    random_key = random.choice(list(row['samples'].keys()))

    user_prompt = f"""
                You are a movie recommender.
                Given the movies/rating pairs from the user below, predict the rating of the movie {row['target'][0]}:
                Return the Prediction in Json Format. The Json should consist of the following fields
                MovieName : The input movie name.
                RatingPrediction : The predicted rating. (Possible Values: 1,2,3,4,5)
                A correct output will look like:
                {{"MovieName" : "{random_key}",
                "RatingPrediction" : "{row['samples'][random_key]}"}}
                There should be no other text, analysis, or code of any kind.
                You will be penalized for every extra token before or after the rating prediction JSON
                <USER INPUT>
                Input:
                Movie Ratings: {sample_string}
                Movie to rate: {row['target'][0]}
                </USER INPUT>
                """
    return system_prompt + '\n' + user_prompt

In [19]:
import re
def parse_result(result):
    match = re.search(r'\{[\s\S]*\}', result)
    if match:
        json_str = match.group(0)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None
    return None

In [20]:
# input_df = pd.read_csv(input_csv_path)
# input_df['id'] = input_df.index.astype(str) + ('_US')+ input_df['userId'].astype(str)
# input_df.to_csv('fin_tst_all.csv', index=False)

In [21]:
from itertools import islice
import ast
import os
import pandas as pd

output_csv_path = f"good/llama70b_rp_10.csv"
bad_csv_path = f"Bad_llama70b_rp_10.csv"
system_prompt = read_system_prompt(system_prompt_file)
input_df = pd.read_csv(input_csv_path)
completed_ids = read_completed_ids(output_csv_path)
bad_ids = read_completed_ids(bad_csv_path)

input_df['samples'] = input_df['samples'].apply(ast.literal_eval)
input_df['target'] = input_df['target'].apply(ast.literal_eval)


input_df = input_df[~input_df['id'].isin(completed_ids)]
input_df = input_df[~input_df['id'].isin(bad_ids)]
print(f"Removed {len(completed_ids)} rows from input dataframe")
print(f"Removed {len(bad_ids)} rows from input dataframe")
print("Remaining rows: ", len(input_df))

i = 0
tot = len(input_df) // 10
print(f"{tot}")
for batch in batch_iterator(input_df.iterrows(), 10):
    i += 1
    if i % 5 == 0:
        print(f"{i} : {tot}")
    batch_prompts = []
    batch_metadata = []
    results = []

    for index, row in batch:
        full_prompt = generate_prompt(row, system_prompt)
        batch_prompts.append(full_prompt)
        this_metadata = {'id': row['id'], 'userId': row['userId'], 'prompt': row['target'], 'samples': row['samples']}
        batch_metadata.append(this_metadata)
        # inputs = tokenizer(full_prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        # with torch.no_grad():
        #     outputs = model.generate(
        #         **inputs,
        #         max_new_tokens=max_new_tokens,
        #         temperature=temperature,
        #         top_p=top_p,
        #         do_sample=True,
        #         pad_token_id=tokenizer.pad_token_id,
        #         stopping_criteria=StoppingCriteriaList([end_curly_brace_criteria])
        #     )
        # results.append(tokenizer.decode(outputs[0])[len(full_prompt):])
        res = llama3(full_prompt) + '}'
        # print(res)
        results.append(res)


    new_rows = []
    bad_rows = []
    for metadata, result in zip(batch_metadata, results):
        # print(result)
        if result is not None:
            # print(result)
            parsed_json = parse_result(result)
            if parsed_json is not None:
                metadata['json'] = parsed_json
                new_rows.append(metadata)
            else:
                # print("couldn't parse this: ", metadata[language_in])
                metadata['result'] = result
                bad_rows.append(metadata)

    if new_rows:
        print(f"Writing {len(new_rows)} rows to csv")
        # print(f"New ids completed: {[row['id'] for row in new_rows]}")
        new_df = pd.DataFrame(new_rows)
        if os.path.exists(output_csv_path):
            new_df.to_csv(output_csv_path, mode='a', header=False, index=False)
        else:
            new_df.to_csv(output_csv_path, index=False)

    if bad_rows:
        bad_df = pd.DataFrame(bad_rows)
        if os.path.exists(bad_csv_path):
            bad_df.to_csv(bad_csv_path, mode='a', header=False, index=False)
        else:
            bad_df.to_csv(bad_csv_path, index=False)




Removed 5930 rows from input dataframe
Removed 0 rows from input dataframe
Remaining rows:  170
17
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
5 : 17
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
10 : 17
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
15 : 17
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
